In [27]:
%pip install pandas
import numpy as np
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [28]:
import matplotlib.pyplot as plt

In [29]:
import sys, os, re, csv, codecs
import tensorflow as tf

In [30]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Conv1D, concatenate, Flatten, Concatenate
from tensorflow.keras.layers import  BatchNormalization, Dropout, Activation, SpatialDropout1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

In [31]:
from sklearn.metrics import hamming_loss
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [32]:
multi_train = pd.read_csv('train.csv')
multi_validation = pd.read_csv('dev.csv')
multi_test = pd.read_csv('test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
multi_train.rename(columns={"Ogling/Facial Expressions/Staring": "Staring", "Touching /Groping": "Touching"}, inplace=True)
multi_validation.rename(columns={"Ogling/Facial Expressions/Staring": "Staring", "Touching /Groping": "Touching"},inplace=True)
multi_test.rename(columns={"Ogling/Facial Expressions/Staring": "Staring", "Touching /Groping": "Touching"},inplace=True)

In [ ]:
multi_train.head()

,Description,Commenting,Staring,Touching
0,"Was walking along crowded street, holding mums...",0,0,1
1,This incident took place in the evening.I was ...,0,1,0
2,I WAS WAITING FOR THE BUS. A MAN CAME ON A BIK...,1,0,0
3,Incident happened inside the train,0,0,0
4,I witnessed an incident when a chain was bruta...,0,0,0


In [ ]:
multi_train.drop_duplicates(keep = 'first', inplace=True)
multi_validation.drop_duplicates(keep = 'first', inplace=True)
multi_test.drop_duplicates(keep = 'first', inplace=True)

In [ ]:
train = pd.concat([multi_train, multi_validation], axis=0 ) # concat train and validation set 
train.head()

,Description,Commenting,Staring,Touching
0,"Was walking along crowded street, holding mums...",0,0,1
1,This incident took place in the evening.I was ...,0,1,0
2,I WAS WAITING FOR THE BUS. A MAN CAME ON A BIK...,1,0,0
3,Incident happened inside the train,0,0,0
4,I witnessed an incident when a chain was bruta...,0,0,0


In [ ]:
test = multi_test  # test set

In [ ]:
train.shape, test.shape

((7735, 4), (1640, 4))

In [ ]:
y_train = train[['Commenting', 'Staring', 'Touching']].values
y_test = test[['Commenting', 'Staring', 'Touching']].values

In [ ]:
# Text Data
 
description_train_text =list(train['Description'].values)
description_test_text = list(test['Description'].values)

# tokenize the Text data
tokenizer = tf.keras.preprocessing.text.Tokenizer()
# fit on train data
tokenizer.fit_on_texts(description_train_text)     
# transform train and test data
train_description_sequences = tokenizer.texts_to_sequences(description_train_text)
test_description_sequences = tokenizer.texts_to_sequences(description_test_text)

# vocabulary size 
vocab_size = len(tokenizer.word_index) + 1

# pad the sequnce

train_description_padded = pad_sequences(
        train_description_sequences, maxlen=300, dtype='int32', padding='post',  
        truncating='post')                                                  

test_description_padded = pad_sequences(
        test_description_sequences, maxlen=300, dtype='int32', padding='post', 
        truncating='post')

print(train_description_sequences)       

[[4, 46, 232, 188, 93, 664, 3284, 169, 14, 74, 1067, 19, 218, 433, 3, 283, 6, 316, 12, 4565, 1, 13, 286, 83, 1, 145, 16, 256, 92, 5, 34, 3, 4, 726, 1017, 124, 67], [24, 97, 81, 68, 7, 2, 49, 3, 4, 7, 2, 84, 14, 72, 48, 33, 96], [3, 4, 179, 25, 2, 29, 5, 19, 56, 11, 5, 100, 1, 4, 2221, 4566, 242, 73], [97, 45, 261, 2, 187], [3, 444, 74, 97, 14, 5, 122, 4, 1961, 175, 20, 74, 1067, 174, 2, 97, 81, 68, 79, 49], [30, 91, 3, 4, 46, 12, 3285, 3, 99, 5, 61, 295, 4567, 173, 440, 10, 5, 32, 239, 32, 637, 165, 239, 4568, 15, 1770, 16, 26, 4569, 6, 2666, 404, 3285], [3, 78, 481, 101, 198, 85, 142, 198, 59], [14, 3, 474, 2, 936, 16, 4, 300, 55, 3, 419, 6, 133, 51, 20, 113, 38, 194, 5, 19, 56, 6, 9, 233, 538, 8, 169, 233, 370, 9, 6, 133, 20, 21, 936, 1, 139, 119, 9, 8, 243, 2222, 317, 936], [131, 318], [429, 93, 399], [210, 1, 86, 59, 17, 72, 10, 2, 4570, 326, 2, 357, 151, 12, 2, 2667, 1771, 420, 9, 1, 8, 35, 220, 16, 39, 3286, 21, 2, 3287, 1, 697, 4571, 12, 2, 1771, 341, 290, 185, 55, 205], [24, 45

In [ ]:
%matplotlib inline 

import nltk
embedding_size = 300  #  size of the embedding vector. 
window_size = 20    # size of the number of words occurring before and after the word based on which the word representations.
min_word = 5  # minimum frequency of a word in the corpus for which the word representations will be generated
down_sampling = 1e-2   # most frequently occurring word will be down-sampled by a number specified by the down_sampling 

word_punctuation_tokenizer = nltk.WordPunctTokenizer()
word_tokenized_corpus = [word_punctuation_tokenizer.tokenize(sent) for sent in description_train_text]



In [ ]:
%%time
ft_model = FastText(word_tokenized_corpus,
                      size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      sg=1,
                      iter=100)

NameError: name 'FastText' is not defined